<a href="https://colab.research.google.com/github/yl4732/TFA-Group-Project/blob/main/v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1tGcX2qzS2GoaN5zFeD5DVJxqDmoQdmMs7QncZwFCEqU/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's totally optional whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_All code should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach._

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [ ]:
# all import statements needed for the project, for example:

import math

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db

In [ ]:
# any general notebook setup, like log formatting

In [ ]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library
* [ ] Write at least one unit test that tests this distance calculation function. 
* Taxi data:
    * [ ] Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box]
* Using the function that calculates the distance between two coordinates in kilometers, add a column to each `pandas` DataFrame of data that calculates the distance between pickup and dropoff locations for each trip.
* Weather data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
        * You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 
* Take a look at the lecture notes from the `pandas` lecture for hints on helpful functionality

### Calculating distance
_Write some prose that tells the reader what you're about to do here._

In [ ]:
def calculate_distance(from_coord, to_coord):
  lat1, lon1 = from_coord
  lat2, lon2 = to_coord
  radius = 6371 # km

  dlat = math.radians(lat2-lat1)
  dlon = math.radians(lon2-lon1)
  a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  d = radius * c
  return d
#calculate_distance((40.776440	, -73.959862), (40.767906, -73.962837))

In [ ]:
def test_calculate_distance():
  assert calculate_distance((40.560445, -74.242330), (40.908524, -73.717047))==58.795089791900814

In [ ]:
from typing import AnyStr
def add_distance_column(dataframe):
  coordinate_distance=[]
  for index, row in dataframe.iterrows():
    from_coord=(row['pickup_latitude'],row['pickup_longitude'])
    to_coord=(row['dropoff_latitude'],row['dropoff_longitude'])
    coordinate_distance.append(calculate_distance(from_coord,to_coord))  
  dataframe= pd.concat([dataframe,pd.DataFrame(coordinate_distance,columns=['coordinate_distance'])], axis=1)
  print(len(coordinate_distance))
  return dataframe


### Processing Taxi Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from bs4 import BeautifulSoup
import re
def find_taxi_csv_urls(TAXI_URL):
    response = requests.get(TAXI_URL)
    if not response.status_code == 200:
        return None
    result_page = BeautifulSoup(response.content,'lxml')
    taxi_urls = list()
    all_taxi_urls=list()

    for year in result_page.find_all('div',{"id":re.compile('faq.*')}):

        for month in year.find_all('a', {"title": 'Yellow Taxi Trip Records'}):
            taxi_urls.append(month.get('href'))

    for i in taxi_urls:
      if i[-11:-4]>="2009-01" and i[-11:-4]<="2015-06":
        all_taxi_urls.append(i)   
    return all_taxi_urls




def get_and_clean_month_taxi_data(url):
    df = pd.read_csv(url)
    df.columns = df.columns.str.lower()
    words = ['time', 'lon', 'lat', 'tip', 'distance','total']
    df_new = pd.DataFrame()
    for i in words:
      col=df.loc[:,df.columns.str.contains(i)]
      df_new=pd.concat([df_new, col], axis=1)
    mapping = {df_new.columns[0]:'pickup_datetime', 
           df_new.columns[1]: 'dropoff_datetime', 
           df_new.columns[2]: 'pickup_longitude', 
           df_new.columns[3]: 'dropoff_longitude',
           df_new.columns[4]: 'pickup_latitude',
           df_new.columns[5]: 'dropoff_latitude',
           df_new.columns[6]: 'tip',
           df_new.columns[7]: 'distance',
           df_new.columns[8]: 'total_amount'}
    yellow_taxi = df_new.rename(columns=mapping)
    yellow_taxi_clean = yellow_taxi.dropna()
    filter = yellow_taxi_clean.loc[yellow_taxi_clean.pickup_longitude.between(-74.242330,-73.717047) & yellow_taxi_clean.dropoff_longitude.between(-74.242330,-73.717047) 
    & yellow_taxi_clean.pickup_latitude.between(40.560445,40.908524)& yellow_taxi_clean.dropoff_latitude.between(40.560445,40.908524)]
    sample= filter.sample(n=2564)
    sample.reset_index(drop=True, inplace=True)
    return sample

    

def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls(TAXI_URL)
    for csv_url in all_csv_urls[:2]:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        re=add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        #save monthly cleaned data into drive
        from pathlib import Path
        #get the month of the cleaned data
        month = csv_url[-11:]
        #create path for the month
        filepath = Path(f'/content/drive/MyDrive/TFA_test/monthly_data/{month}') 
        #check whether there is an existing parent path:
        #if there is, save file under the path; if there isn't, create a new parent path
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        re.to_csv(filepath, index=False)  
        all_taxi_dataframes.append(re)

    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    
    #save the aggregate cleaned data into drive
    filepath = Path('/content/drive/MyDrive/TFA_test/taxi.csv')  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    taxi_data.to_csv(filepath, index=False)  
    return taxi_data

In [ ]:
get_and_clean_taxi_data()

2564
2564


,pickup_datetime,dropoff_datetime,pickup_longitude,dropoff_longitude,pickup_latitude,dropoff_latitude,tip,distance,total_amount,coordinate_distance
0,2015-01-30 04:04:05,2015-01-30 04:09:55,-73.990761,-73.988037,40.761074,40.774895,1.00,1.30,8.80,1.553809
1,2015-01-24 10:57:38,2015-01-24 11:01:28,-73.982292,-73.993599,40.767464,40.767456,1.70,0.90,7.50,0.952202
2,2015-01-12 23:32:16,2015-01-12 23:41:49,-74.004448,-73.977867,40.741833,40.746510,2.50,2.23,13.30,2.298886
3,2015-01-26 13:47:44,2015-01-26 14:02:43,-73.961525,-73.978668,40.764145,40.782700,2.00,2.30,14.30,2.518075
4,2015-01-17 16:07:23,2015-01-17 16:26:36,-73.962120,-73.969826,40.778973,40.752274,0.00,2.34,14.30,3.038901
...,...,...,...,...,...,...,...,...,...,...
2559,2015-02-15 01:09:32,2015-02-15 01:26:51,-73.990044,-73.945816,40.733906,40.707806,4.20,4.38,21.00,4.723895
2560,2015-02-19 11:13:33,2015-02-19 11:30:01,-73.994690,-73.993462,40.743736,40.752785,0.00,0.80,11.80,1.011450
2561,2015-02-14 01:24:41,2015-02-14 01:26:50,-73.880737,-73.883804,40.748032,40.747128,0.00,0.20,4.30,0.277236
2562,2015-02-23 21:21:34,2015-02-23 21:36:14,-74.000328,-73.978104,40.718369,40.663528,3.26,4.30,19.56,6.379338


In [ ]:
pd.read_csv('/content/drive/MyDrive/TFA_test/monthly_data/2015-01.csv')

,pickup_datetime,dropoff_datetime,pickup_longitude,dropoff_longitude,pickup_latitude,dropoff_latitude,tip,distance,total_amount,coordinate_distance
0,2015-01-30 04:04:05,2015-01-30 04:09:55,-73.990761,-73.988037,40.761074,40.774895,1.0,1.30,8.8,1.553809
1,2015-01-24 10:57:38,2015-01-24 11:01:28,-73.982292,-73.993599,40.767464,40.767456,1.7,0.90,7.5,0.952202
2,2015-01-12 23:32:16,2015-01-12 23:41:49,-74.004448,-73.977867,40.741833,40.746510,2.5,2.23,13.3,2.298886
3,2015-01-26 13:47:44,2015-01-26 14:02:43,-73.961525,-73.978668,40.764145,40.782700,2.0,2.30,14.3,2.518075
4,2015-01-17 16:07:23,2015-01-17 16:26:36,-73.962120,-73.969826,40.778973,40.752274,0.0,2.34,14.3,3.038901
...,...,...,...,...,...,...,...,...,...,...
2559,2015-01-22 06:28:01,2015-01-22 06:33:10,-73.985596,-73.973907,40.746777,40.738548,0.0,0.99,6.3,1.344154
2560,2015-01-01 03:20:21,2015-01-01 03:29:38,-73.981796,-73.973862,40.773380,40.794209,0.0,2.19,10.3,2.410420
2561,2015-01-03 01:19:36,2015-01-03 01:27:40,-73.970566,-73.948929,40.755997,40.783283,2.0,2.30,12.3,3.539204
2562,2015-01-28 17:30:20,2015-01-28 17:44:16,-73.966820,-73.952919,40.761513,40.742374,1.7,2.80,16.0,2.428955


In [ ]:
ans.columns = ans.columns.str.lower()
words = ['time', 'lon', 'lat', 'tip', 'distance','total']
df = pd.DataFrame()
for i in words:
  col=ans.loc[:,ans.columns.str.contains(i)]
  df=pd.concat([df, col], axis=1)
df  
#ans.loc[:,[ans.columns.str.contains(i) for i in words]]


In [ ]:
#find_taxi_csv_urls(TAXI_URL)

In [ ]:
mapping = {df.columns[0]:'pickup_datetime', 
           df.columns[1]: 'dropoff_datetime', 
           df.columns[2]: 'pickup_longitude', 
           df.columns[3]: 'dropoff_longitude',
           df.columns[4]: 'pickup_latitude',
           df.columns[5]: 'dropoff_latitude',
           df.columns[6]: 'tip',
           df.columns[7]: 'distance',
           df.columns[8]: 'total_amount'}
yellow_taxi = df.rename(columns=mapping)

In [ ]:
yellow_taxi.isnull().sum()
yellow_taxi_clean = yellow_taxi.dropna()

In [ ]:
filter = yellow_taxi_clean.loc[yellow_taxi_clean.pickup_longitude.between(-74.242330,-73.717047) 
& yellow_taxi_clean.dropoff_longitude.between(-74.242330,-73.717047) 
& yellow_taxi_clean.pickup_latitude.between(40.560445,40.908524)
& yellow_taxi_clean.dropoff_latitude.between(40.560445,40.908524)
]

In [ ]:
filter

In [ ]:
filter.sample(n=2564)

### Processing Uber Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
def load_and_clean_uber_data(csv_file):
  df = pd.read_csv(csv_file)
  df=df.drop(columns=['Unnamed: 0','key','passenger_count'])
  df = df.dropna(how='any')
  df = df.loc[df.pickup_longitude.between(-74.242330,-73.717047) & df.dropoff_longitude.between(-74.242330,-73.717047) 
    & df.pickup_latitude.between(40.560445,40.908524)& df.dropoff_latitude.between(40.560445,40.908524)]
  df.reset_index(drop=True, inplace=True)
  return df

def get_uber_data():
  uber_dataframe = load_and_clean_uber_data(UBER_CSV)
  df=add_distance_column(uber_dataframe)
  return df
get_uber_data()

195472


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,coordinate_distance
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1.683323
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,2.457590
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,5.036377
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,1.661683
4,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,4.475450
...,...,...,...,...,...,...,...
195467,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,0.112210
195468,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1.875050
195469,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,12.850319
195470,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,3.539715


### Processing Weather Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    we = pd.read_csv(csv_file)
    f1 = ['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']
    we_1 = pd.DataFrame()
    for i in f1:
        col1=we.loc[:,we.columns.str.contains(i)]
        we_1=pd.concat([we_1, col1], axis=1)
    hourly_clean= we_1.dropna()
    hourly_clean.reset_index(drop=True, inplace=True)
    return hourly_clean

def clean_month_weather_data_daily(csv_file):
    we = pd.read_csv(csv_file)
    f2 = ['DATE', 'DailySustainedWindSpeed']
    we_2 = pd.DataFrame()
    for i in f2:
        col2=we.loc[:,we.columns.str.contains(i)]
        we_2=pd.concat([we_2, col2], axis=1)
    daily_clean= we_2.dropna()
    daily_clean.reset_index(drop=True, inplace=True)
    return daily_clean
    

def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["2009_weather.csv", 
                         "2010_weather.csv", 
                         "2011_weather.csv",
                         "2012_weather.csv",
                         "2013_weather.csv",
                         "2014_weather.csv",
                         "2015_weather.csv"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    return hourly_data, daily_data

In [ ]:
load_and_clean_weather_data() 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DtypeWarning: Columns (8,9,10,17) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DtypeWarning: Columns (8,9,10,17) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory

(                     DATE HourlyPrecipitation  HourlyWindSpeed
 0     2009-01-02T12:34:00                   T             14.0
 1     2009-01-02T12:51:00                   T             11.0
 2     2009-01-02T13:05:00                   T              7.0
 3     2009-01-02T13:30:00                   T              9.0
 4     2009-01-02T13:46:00                   T             10.0
 ...                   ...                 ...              ...
 9102  2015-12-31T18:51:00                0.00              3.0
 9103  2015-12-31T19:51:00                0.00              6.0
 9104  2015-12-31T20:51:00                0.00             10.0
 9105  2015-12-31T22:51:00                0.00              7.0
 9106  2015-12-31T23:51:00                0.00              5.0
 
 [38260 rows x 3 columns],                     DATE  DailySustainedWindSpeed
 0    2012-07-31T23:59:00                     10.0
 1    2012-08-01T23:59:00                      9.0
 2    2012-08-02T23:59:00                     12.0


### Process All Data

_This is where you can actually execute all the required functions._

_Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather, daily_weather = load_and_clean_weather_data()

NameError: ignored

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    DATE TEXT,
    HourlyPrecipitation TEXT,
    HourlyWindSpeed INTEGER
)
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    DATE TEXT,
    DailySustainedWindSpeed INTEGER
)
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_Write some prose that tells the reader what you're about to do here._

In [ ]:
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

In [ ]:
def dataframes_to_table(table_name_to_dataframe, engine):
  for key in table_name_to_dataframe:
    table_name_to_dataframe[key].to_sql(key, con=engine, if_exists="replace", index=False)

In [ ]:
table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

dataframes_to_table(table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.
* [ ] Come up with 3 questions on your own that can be answered based on the data in the 4 tables. Create at least one visualization to answer each question. At least one visualization should require data from at least 3 tables.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_You don't have to query the data directly from the database. You can just re-use the pandas DataFrame that you created in Part 1._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
plot_visual_n(some_dataframe)